In [1]:
import os
import shutil
import pickle
import numpy as np

In [2]:
def convert(box):
    x, y, w, h = box
    bbox = [x, y, x + w, y + h]

    return bbox

def make_labels(clip_path):
    label_path = f'{clip_path}/labels'

    if os.path.exists(label_path):
        shutil.rmtree(label_path)
    os.makedirs(label_path)

    ball_ids = []
    goalkeeper_ids = []
    referee_ids = []

    with open(f'{clip_path}/gameinfo.ini', 'r') as file:
        data = file.readlines()
        for line in data:
            if 'ball' in line:
                ball_ids.append(eval(line.split('=')[0].split('_')[1]))
            if 'goalkeeper' in line:
                goalkeeper_ids.append(eval(line.split('=')[0].split('_')[1]))
            if 'referee' in line:
                referee_ids.append(eval(line.split('=')[0].split('_')[1]))
            
    with open(f'{clip_path}/gt/gt.txt', 'r') as file:
        data = file.readlines()
        for line in data:
            line = [eval(i) for i in line.split(',')]
            bbox = convert(line[2:6])

            class_id = 1
            if line[1] in ball_ids:
                class_id = 0
            elif line[1] in goalkeeper_ids:
                class_id = 2
            elif line[1] in referee_ids:
                class_id = 3

            with open(
                f'{label_path}/{line[0]}.txt', 'a'
            ) as f:
                f.write(f'{class_id} {' '.join([str(i) for i in bbox])}\n')

In [3]:
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    
    x5, y5, x6, y6 = max(x1, x3), max(y1, y3), min(x2, x4), min(y2, y4)
    if x5 >= x6 or y5 >= y6:
        return 0
    
    intersection = (x6 - x5) * (y6 - y5)
    union = (x2 - x1) * (y2 - y1) + (x4 - x3) * (y4 - y3) - intersection
    
    return intersection / union

def mAP(pkl_path, label_path, iou_threshold=0.5):
    with open(pkl_path, 'rb') as file:
        detections, _ = pickle.load(file)
    
    aps = []
    for class_id in range(4):
        tp = []
        fp = []
        scores = []
        num_annotations = 0

        for i, det in enumerate(detections):
            det = det[det.class_id == class_id]

            labels = []
            with open(f'{label_path}/{i + 1}.txt', 'r') as f:
                data = f.readlines()

                for line in data:
                    line = line.split(' ')

                    if eval(line[0]) == class_id:
                        bbox = [eval(i) for i in line[1:]]
                        labels.append(bbox)
            
            if len(labels) == 0:
                continue

            num_annotations += len(labels)

            ious = []
            for box in det.xyxy:
                ious.append(max([iou(box, label) for label in labels]))
            
            for j, box in enumerate(det):
                scores.append(det.confidence[j])
                if ious[j] > iou_threshold:
                    tp.append(1)
                    fp.append(0)
                else:
                    tp.append(0)
                    fp.append(1)

        if num_annotations == 0:
            aps.append(0)
            continue

        tp = np.array(tp)
        fp = np.array(fp)
        scores = np.array(scores)

        indices = np.argsort(-scores)
        tp = tp[indices]
        fp = fp[indices]

        tp = np.cumsum(tp)
        fp = np.cumsum(fp)

        recalls = tp / num_annotations
        precisions = tp / (tp + fp)

        recalls = np.concatenate(([0.0], recalls, [1.0]))
        precisions = np.concatenate(([0.0], precisions, [0.0]))

        for i in range(precisions.size - 1, 0, -1):
            precisions[i - 1] = np.maximum(precisions[i - 1], precisions[i])

        indices = np.where(recalls[1:] != recalls[:-1])[0]
        ap = np.sum((recalls[indices + 1] - recalls[indices]) * precisions[indices + 1])
        aps.append(ap)

    return aps

In [8]:
clips = [116, 117, 118, 120, 130]

for clip in clips:
    make_labels(f'datasets/test/SNMOT-{clip}')

In [9]:
thresholds = [0.5, 0.75, 0.9]

for threshold in thresholds:
    print(f'Threshold: {threshold}')

    for clip in clips:
        print(mAP(f'datasets/test/SNMOT-{clip}/det.pkl', f'datasets/test/SNMOT-{clip}/labels', threshold))
    print()

Threshold: 0.5
[0.020590798474085764, 0.8054181127643385, 0.06312345765949474, 0.8439177998731349]
[0.07235674244783556, 0.8849176022146755, 0.6247429179107079, 0.9140042453001456]
[0.1221324269680029, 0.9148462069640513, 0.7101166398434593, 0.8763493321519991]
[0.1334016741237702, 0.8879411358356383, 0.7269194536033914, 0.7758627618420904]
[0.14730033137182175, 0.9487244835302379, 0.7371555814358132, 0.9959340941722329]

Threshold: 0.75
[0.0, 0.5854449801896118, 0.04182427181303299, 0.6548588695351834]
[0.001200300954640948, 0.7378232591537367, 0.5116694545365589, 0.660953041306512]
[0.029548821364381365, 0.6477611002405286, 0.47870148106899035, 0.5990962514306335]
[0.017747346799041427, 0.7591970764448509, 0.6134112079425049, 0.5683485495910152]
[0.06518619921240762, 0.822277054755705, 0.5641638763083398, 0.9578480455800008]

Threshold: 0.9
[0.0, 0.08802033519362865, 0.00929679750581962, 0.12512156810487868]
[0.0, 0.13751574282403206, 0.03204342273782317, 0.1051169913808003]
[0.00041